In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
torch.set_num_threads(8)

# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            # 'dataset': "CIFAR100",
            'dataset': "MNIST",
            'random_class_idx': True,

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 5,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 1,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.BCEWithLogitsLoss(),
            'device': torch.device('cuda:5' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.000,
            'lambda': .0,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=MNIST/seed=10_epoch=5_lr=0.001_alpha=0.0_tau=1


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                    per_task_memory_examples=params['per_task_memory_examples'],
                    per_task_examples = params['per_task_examples'],
                    random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)



[8 2 5 6 3 1 0 7 4 9]


In [3]:
from algorithms.icarl import iCaRL
from metrics import MetricCollector2
from backbones import MLP2Layers2

backbone = MLP2Layers2(
    input_dim=input_dim, 
    hidden_dim_1=256, 
    hidden_dim_2=256, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])

# from backbones import ResNet18
# backbone = ResNet18(
#     input_dim=input_dim, 
#     output_dim=num_classes,
#     class_idx=class_idx,
#     config=params
#     ).to(params['device'])

algorithm = iCaRL(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [4]:
metric_manager_callback.meters['accuracy'].data.shape

(6, 6, 5)

In [5]:
from trainers.baselines import BaseMemoryContinualTrainer as ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [6]:
# CIFAR10: self.dataset.data.shape=(50000, 32, 32, 3)


In [7]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self

Exception ignored in: <function _releaseLock at 0x7fb3883df8b0>
Traceback (most recent call last):
  File "/home/jaeyoung/anaconda3/envs/cil/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset.data.shape=torch.Size([60000, 28, 28])
self.dataset

In [ ]:
metric_manager_callback.meters['EER'].get_eer()

[0.0,
 0.0029377586790896304,
 0.02922144834681062,
 0.04689474638930541,
 0.056992683670983445]

In [ ]:
metric_manager_callback.meters['std'].get_std()

[0.03727138150040593,
 0.03517587226968666,
 0.07032850022082138,
 0.08288781294947618,
 0.09570917188363709]

In [ ]:
metric_manager_callback.meters['EER'].get_eer()

[0.0,
 0.0029377586790896304,
 0.02922144834681062,
 0.04689474638930541,
 0.056992683670983445]

In [ ]:
metric_manager_callback.meters['std'].get_std()

[0.03727138150040593,
 0.03517587226968666,
 0.07032850022082138,
 0.08288781294947618,
 0.09570917188363709]

In [ ]:
metric_manager_callback.meters['accuracy'].get_data()

array([[0.944, 0.   , 0.   , 0.   , 0.   ],
       [0.881, 0.875, 0.   , 0.   , 0.   ],
       [0.832, 0.829, 0.896, 0.   , 0.   ],
       [0.796, 0.813, 0.894, 0.902, 0.   ],
       [0.745, 0.797, 0.889, 0.875, 0.748]])

In [ ]:
metric_manager_callback.meters['accuracy'].compute_overall()

0.8672964562143524

In [ ]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"EER:{np.mean(metric_manager_callback.meters['EER'].compute_overall())}")
print(f"std:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")


accuracy:0.8672964562143524
EER:0.027209327417237823
std:0.06427454776480544
